In [28]:
import os
import chromadb
import pypdf
import numpy as np
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer


In [ ]:
# 初始化 ChromaDB
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.get_or_create_collection(name="pdf_documents")

# 選擇適合中文的語意檢索模型
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

In [ ]:
def extract_text_from_pdf(pdf_path):
    """ 從 PDF 提取文字內容 """
    text = ""
    with open(pdf_path, "rb") as file:
        reader = pypdf.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text() + "\n"
    return text

In [ ]:
def split_text(text, chunk_size=500, chunk_overlap=50):
    """ 切分文本為小段落 """
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return splitter.split_text(text)

In [ ]:
def text_to_vector(text):
    """ 將文本轉換為向量 """
    return model.encode(text).tolist()

In [ ]:
def store_vectors_in_chroma(text_chunks, file_name):
    """ 儲存文本向量到 ChromaDB，並包含文件名稱 """
    for idx, chunk in enumerate(text_chunks):
        collection.add(
            ids=[f"{file_name}_{idx}"],
            embeddings=[text_to_vector(chunk)],
            metadatas=[{"text": chunk, "file_name": file_name}]
        )

In [ ]:
def process_pdfs(pdf_folder):
    """ 處理指定資料夾內的所有 PDF，並存入向量資料庫 """
    for file in os.listdir(pdf_folder):
        if file.endswith(".pdf"):
            pdf_path = os.path.join(pdf_folder, file)
            print(f"處理文件: {file}")
            pdf_text = extract_text_from_pdf(pdf_path)
            text_chunks = split_text(pdf_text)
            store_vectors_in_chroma(text_chunks, file)

In [ ]:
def search_chroma(query, top_k=3):
    """ 使用 ChromaDB 查詢相似文本，並回傳對應的 PDF 文件名稱 """
    query_vector = text_to_vector(query)
    results = collection.query(query_embeddings=[query_vector], n_results=top_k)
    
    matched_texts = [r["text"] for r in results["metadatas"][0]]
    file_names = [r["file_name"] for r in results["metadatas"][0]]
    
    return list(zip(file_names, matched_texts))

In [ ]:
# 🔹 處理多個 PDF 文件
process_pdfs("D:\.vscode\專題\product infomation")  # 你的 PDF 資料夾路徑

處理文件: DE-291-1 DE-293 工作桌.pdf


Insert of existing embedding ID: DE-291-1 DE-293 工作桌.pdf_0
Add of existing embedding ID: DE-291-1 DE-293 工作桌.pdf_0
Insert of existing embedding ID: SADES DIABLO 暗黑鬥狼RGB REALTEK 電競耳麥 7.1 (USB) SA-916.pdf_0
Add of existing embedding ID: SADES DIABLO 暗黑鬥狼RGB REALTEK 電競耳麥 7.1 (USB) SA-916.pdf_0


處理文件: SADES DIABLO 暗黑鬥狼RGB REALTEK 電競耳麥 7.1 (USB) SA-916.pdf


Insert of existing embedding ID: SADES DIABLO 暗黑鬥狼RGB REALTEK 電競耳麥 7.1 (USB) SA-916.pdf_1
Add of existing embedding ID: SADES DIABLO 暗黑鬥狼RGB REALTEK 電競耳麥 7.1 (USB) SA-916.pdf_1
Insert of existing embedding ID: W202 人體工學椅.pdf_0
Add of existing embedding ID: W202 人體工學椅.pdf_0
Insert of existing embedding ID: [折疊收納]懶人折疊桌.pdf_0
Add of existing embedding ID: [折疊收納]懶人折疊桌.pdf_0


處理文件: W202 人體工學椅.pdf
處理文件: [折疊收納]懶人折疊桌.pdf
處理文件: 羅技 Logitech H340 USB耳機麥克風.pdf


Insert of existing embedding ID: 羅技 Logitech H340 USB耳機麥克風.pdf_0
Add of existing embedding ID: 羅技 Logitech H340 USB耳機麥克風.pdf_0
Insert of existing embedding ID: 羅技 Logitech H340 USB耳機麥克風.pdf_1
Add of existing embedding ID: 羅技 Logitech H340 USB耳機麥克風.pdf_1


In [ ]:
# 🔹 測試查詢
query = "辦公桌"
search_results = search_chroma(query, 5)
for i, (file_name, result) in enumerate(search_results):
    print(f"來自文件: {file_name}\n相關段落 {i+1}: {result}\n")

來自文件: [折疊收納]懶人折疊桌.pdf
相關段落 1: 商品名稱[折疊收納]懶人折疊桌 店家名稱吉來小舖
商品售價95 店家電話未提供
店家地址未提供
商品描述
商品資料表
產品名稱：懶人折疊桌
產品顏色：木紋
展開尺寸：50x30x23cm
折疊尺寸：50x30x5cm
產品重量：約1290克
產品材質：密度板+鋼管+塑料
產品用途：折疊桌
組裝需求：無
使用方法：展開即可使用
產品產地：中國
產品貨源：原廠公司貨
保存方法：本產品請避光陰涼處存放。
產品內容：懶人折疊桌x1

來自文件: W202 人體工學椅.pdf
相關段落 2: 商品名稱W202 人體工學椅 店家名稱Xpanse旗艦店
商品售價1,460 店家電話未提供
店家地址未提供
商品描述
商品資料表
耐重粗鐵管腳+強化塑料框架 結構穩實乘坐安全可靠
椅背採優選透氣網布，韌性高易拉伸不易斯裂
高密度定型棉坐墊，柔軟舒適久坐
當空間較小時或需搬動時可以瞬間收納，不佔空間
㇐體式折合桌椅輕便重量只有8KG,收納移動簡單便利 
已投保1000萬-產品責任險~讓您安心使用多㇐分保障
有實體店面，購買有保障
【規格】
材質：網布，泡棉，PP塑料，鋼管
收合尺寸：寬62CM*高100CM*厚度25CM(詳細尺寸如圖所示 手量尺寸正負2CM)
展開尺寸：(詳細尺寸如圖所示 手量尺寸正負2CM)
商品重量：8KG(單張) 
組裝：【免組裝】，圖片為組裝後之成品。

來自文件: 羅技 Logitech H340 USB耳機麥克風.pdf
相關段落 3: 商品名稱羅技 Logitech H340 USB耳機麥克風 店家名稱筑蒂資訊有限公司
商品售價945 店家電話04-22216131
店家地址台灣省台中市中山路49巷11-2號
商品描述
商品資料表
H340 USB Computer Headset
UBS隨插即用
H340 USB 耳機麥克風具有簡易隨插即用 USB-A 
連線功能，是完美的日常工作耳機麥克風。隔噪麥克風可減少背景噪音，為 VOIP 和 通話提供清晰的數位音訊。
隔噪麥克風
右側的麥克風可調整位置以獲得更好的收音效果，並隔離背景雜音。麥克風桿不使用時可以將其完全折入頭帶中。
可調整頭帶
配備旋轉安裝式泡棉耳墊的輕盈頭帶，讓您⾧時間使用也能享有舒適的音訊聆聽感受。
USB-A 連線
USB 提供簡單

In [ ]:
with open('a.txt', 'w', encoding='utf-8') as f:
    for i, (file_name, result) in enumerate(search_results):
        f.write(f"來自文件: {file_name}\n相關段落 {i+1}: {result}\n\n")